# Que pasa si los datos son muy grandes?
Si los datos a extraer de la base de datos son muy grandes, y al final solo se utilizan una pequeña parte de ellos, se puede hacer una selección de los datos a extraer, para que el tiempo de ejecución sea menor.

~~~bash
pip install pandas-gbq -U
~~~

## 1. Query Personalizada
Al extraer los datos de una base de datos, podemos ejecutar una query que este formateada para extraer los datos que necesitamos.

In [4]:
import pandas as pd

enero_2024 = pd.read_csv("data/bigquery_hist_sales/Sales_2024_1.csv")

In [16]:
enero_2024.columns

Index(['Product', 'Quantity Ordered', 'Price Each', 'Order Date',
       'Purchase Address', 'Category', 'city', 'sales'],
      dtype='object')

In [22]:
enero_2024['city'] = enero_2024['Purchase Address'].apply(lambda x: x.split(', ')[1])
enero_2024['sales'] = enero_2024['Quantity Ordered'] * enero_2024['Price Each']
enero_resumen = enero_2024.groupby('city')['sales'].sum().reset_index()
enero_resumen['promedio'] = enero_resumen['sales'].mean()
stores_analisis = enero_resumen[enero_resumen['sales'] < enero_resumen['promedio']]
stores_analisis_list = str(stores_analisis['city'].tolist()).replace('[','(').replace(']',')')
stores_analisis_list

"('Atlanta', 'Austin', 'Dallas', 'Portland', 'Seattle')"

In [23]:
from google.oauth2 import service_account
import pandas_gbq

credentials = service_account.Credentials.from_service_account_file(
    'secretos/credentials_bigquery.json',
)

In [24]:
year = 2024
month = 2

query = f"""
SELECT * 
FROM `cogent-tree-369319.codigo_facilito_dw.sales_report_hist` 
WHERE EXTRACT(YEAR FROM CAST(order_date AS DATE)) = {year} 
AND EXTRACT(MONTH FROM CAST(order_date AS DATE)) = {month}
AND SPLIT(purchase_address, ', ')[SAFE_OFFSET(1)] IN {stores_analisis_list}
"""
print(query)


SELECT * 
FROM `cogent-tree-369319.codigo_facilito_dw.sales_report_hist` 
WHERE EXTRACT(YEAR FROM CAST(order_date AS DATE)) = 2024 
AND EXTRACT(MONTH FROM CAST(order_date AS DATE)) = 2
AND SPLIT(purchase_address, ', ')[SAFE_OFFSET(1)] IN ('Atlanta', 'Austin', 'Dallas', 'Portland', 'Seattle')



In [25]:
df = pandas_gbq.read_gbq(query, project_id="cogent-tree-369319", credentials=credentials)
df

Downloading: 100%|██████████|


,product,quantity_ordered,price_each,order_date,purchase_address,category
0,AA Batteries (4-pack),4,3.84,2024-02-27,"218 West St, Seattle, WA 98101",Pilas
1,AA Batteries (4-pack),4,3.84,2024-02-25,"847 Center St, Portland, OR 97035",Pilas
2,AA Batteries (4-pack),4,3.84,2024-02-27,"933 Wilson St, Seattle, WA 98101",Pilas
3,AA Batteries (4-pack),4,3.84,2024-02-29,"493 Church St, Dallas, TX 75001",Pilas
4,AA Batteries (4-pack),4,3.84,2024-02-29,"43 Church St, Atlanta, GA 30301",Pilas
...,...,...,...,...,...,...
3596,AAA Batteries (4-pack),3,2.99,2024-02-08,"115 6th St, Atlanta, GA 30301",Pilas
3597,AAA Batteries (4-pack),3,2.99,2024-02-03,"165 Chestnut St, Seattle, WA 98101",Pilas
3598,AAA Batteries (4-pack),3,2.99,2024-02-15,"368 Spruce St, Dallas, TX 75001",Pilas
3599,AAA Batteries (4-pack),3,2.99,2024-02-15,"329 Washington St, Dallas, TX 75001",Pilas


El límite de parámetros que puedes enviar a diferentes bases de datos SQL utilizando una query desde Python depende de varios factores, incluyendo:

La base de datos específica:

- MySQL: Tiene un límite de 65,535 parámetros en una consulta preparada.
- PostgreSQL: El límite es de 32,767 parámetros por consulta.
- SQL Server: Permite hasta 2100 parámetros en una sola consulta.
- Oracle: El límite puede variar según la versión y configuración, pero generalmente es de alrededor de 1000 parámetros.
- SQLite: No tiene un límite fijo en la cantidad de parámetros, pero está limitado por los recursos del sistema.

## 2. Creando una tabla en la base de datos
Podemos crear una tabla en la base de datos y luego hacer un join con la tabla original.
Este metodo es el mas eficiente, pero requiere permisos para crear tablas en la base de datos.

In [ ]:
df_stores = stores_analisis[['city']]
table_id = "codigo_facilito_dw.stores_fil"
project_id="cogent-tree-369319"
pandas_gbq.to_gbq(df_stores, table_id, project_id=project_id, if_exists='replace')

100%|██████████| 1/1 [00:00<?, ?it/s]


In [28]:
query = f"""
SELECT * 
FROM `cogent-tree-369319.codigo_facilito_dw.sales_report_hist` srh
INNER JOIN `cogent-tree-369319.codigo_facilito_dw.stores_fil` sf 
    ON SPLIT(purchase_address, ', ')[SAFE_OFFSET(1)] = sf.city
WHERE EXTRACT(YEAR FROM CAST(srh.order_date AS DATE)) = {year} 
AND EXTRACT(MONTH FROM CAST(srh.order_date AS DATE)) = {month}
"""
print(query)


SELECT * 
FROM `cogent-tree-369319.codigo_facilito_dw.sales_report_hist` srh
INNER JOIN `cogent-tree-369319.codigo_facilito_dw.stores_fil` sf 
    ON SPLIT(purchase_address, ', ')[SAFE_OFFSET(1)] = sf.city
WHERE EXTRACT(YEAR FROM CAST(srh.order_date AS DATE)) = 2024 
AND EXTRACT(MONTH FROM CAST(srh.order_date AS DATE)) = 2



In [29]:
df = pandas_gbq.read_gbq(query, project_id, credentials=credentials)

Downloading: 100%|██████████|


In [30]:
df

,product,quantity_ordered,price_each,order_date,purchase_address,category,city
0,AA Batteries (4-pack),4,3.84,2024-02-27,"218 West St, Seattle, WA 98101",Pilas,Seattle
1,AA Batteries (4-pack),4,3.84,2024-02-25,"847 Center St, Portland, OR 97035",Pilas,Portland
2,AA Batteries (4-pack),4,3.84,2024-02-27,"933 Wilson St, Seattle, WA 98101",Pilas,Seattle
3,AA Batteries (4-pack),4,3.84,2024-02-29,"493 Church St, Dallas, TX 75001",Pilas,Dallas
4,AA Batteries (4-pack),4,3.84,2024-02-29,"43 Church St, Atlanta, GA 30301",Pilas,Atlanta
...,...,...,...,...,...,...,...
3596,AAA Batteries (4-pack),3,2.99,2024-02-08,"115 6th St, Atlanta, GA 30301",Pilas,Atlanta
3597,AAA Batteries (4-pack),3,2.99,2024-02-03,"165 Chestnut St, Seattle, WA 98101",Pilas,Seattle
3598,AAA Batteries (4-pack),3,2.99,2024-02-15,"368 Spruce St, Dallas, TX 75001",Pilas,Dallas
3599,AAA Batteries (4-pack),3,2.99,2024-02-15,"329 Washington St, Dallas, TX 75001",Pilas,Dallas
